In [3]:
# import libraies
import numpy as np 
import pandas as pd
import bs4
import requests
import re
import time

In [4]:
# # read skills data (list of skills Calibri)
# link : https://docs.google.com/spreadsheets/d/18URpIWCjQuKu6G0j3vPUQjsuFKjq_qiMXZ1Xp3ZK_G0/edit?usp=sharing
# skills_df = pd.read_csv('skills.csv', sep=';')
# skills_df.head(10)
# skills = skills_df['skill']


In [5]:
# But we will use those for testing purposes
skills = ['communication', 'creativity', 'collaboration', 'self awareness', 
          'emotional intelligence', 'planning and organization', 'positive psychology']
          
criteria = ['common issues', 'solution for common issues', 'general guidelines']

In [6]:
# The main page of wikihow contains search bar where we can look for what we want
# this function takes a string as input and return the list of links results

def search_wikihow(words):
    url = 'https://www.wikihow.com/wikiHowTo?search=' + words
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, "html.parser")
    links = []
    
    x = soup.select('.result_link') # results are in <a> tags of class results_link, get all fo them

    for i in range(len(x)): # for each of theses <a> tags we extract the adress of the link (href)
        links.append(x[i].get('href'))
    return links

In [7]:
# this function extrat, for link, the body tag and the title of the page.
# The other two parameter, skill_name and criteria_name can be used in another version to
# filter out links whose body doesn't coontain any word contained in skill_name or criteria_name

# This header is used so that the website don't block the request (to be checked if we have to do it for wikihow)
headers_ = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)'
'AppleWebKit 537.36 (KHTML, like Gecko) Chrome',
'Accept':'text/html,application/xhtml+xml,application/xml;'
'q=0.9,image/webp,*/*;q=0.8'}


def extract_body_tag(links_wiki, skill_name, criteria_name) : 
    links_df = [] # links of the pages
    titles_df = [] # title before the tage
    bodies_df = []
    skill_name_df = []
    criteria_name_df = []

    for link in links_wiki:
        session = requests.Session()
        res = session.get(link, headers=headers_)
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        skill_name_df.append(skill_name)
        criteria_name_df.append(criteria_name)
        links_df.append(link)
        titles_df.append(soup.title.get_text())
        bodies_df.append(soup.body) #bodies_df.append(soup.body.get_text())

    text_df = pd.DataFrame.from_dict({'Soft Skill Name' : skill_name_df, 'Criteria':criteria_name_df,'URL' : links_df, 'Title of URL' : titles_df, 'Content' : bodies_df})
    return text_df



In [8]:
# create the empty dataframe that will contains the scraped data. for each link, the df containing the scraped data
# will be concatained with this one
wikihow_df = pd.DataFrame.from_dict({'Soft Skill Name' : [], 'Criteria':[],'URL' : [], 'Title of URL' : [], 'Content' : []})

for skill in skills:
  for c in criteria:
    links = search_wikihow(skill + ' ' + c)
    link_df = extract_body_tag(links, skill, c)
    wikihow_df = pd.concat([wikihow_df, link_df])

wikihow_df.info()
wikihow_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 14
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Soft Skill Name  310 non-null    object
 1   Criteria         310 non-null    object
 2   URL              310 non-null    object
 3   Title of URL     310 non-null    object
 4   Content          310 non-null    object
dtypes: object(5)
memory usage: 14.5+ KB


,Soft Skill Name,Criteria,URL,Title of URL,Content
0,communication,common issues,https://www.wikihow.com/Deal-With-Common-Kissi...,3 Ways to Deal With Common Kissing Issues - wi...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
1,communication,common issues,https://www.wikihow.com/Fix-Common-Computer-Ne...,3 Ways to Fix Common Computer Network Issues -...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
2,communication,common issues,https://www.wikihow.com/Teach-Your-Child-Good-...,3 Ways to Teach Your Child Good Interpersonal ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
3,communication,common issues,https://www.wikihow.com/Overcome-Trust-Issues-...,How to Overcome Trust Issues in a Relationship...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
4,communication,common issues,https://www.wikihow.com/Develop-Good-Communica...,How to Develop Good Communication Skills (with...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
...,...,...,...,...,...
10,positive psychology,general guidelines,https://www.wikihow.com/Pass-a-Psychological-T...,3 Ways to Pass a Psychological Test for a Job ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
11,positive psychology,general guidelines,https://www.wikihow.com/Write-a-Psychological-...,How to Write a Psychological Horror Story: 12 ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
12,positive psychology,general guidelines,https://www.wikihow.com/Category:Psychology-Ca...,Psychology Careers - how to articles from wikiHow,window.a=window.a||{};window.a.f=function(){fu...
13,positive psychology,general guidelines,https://www.wikihow.com/What-Are-the-Psycholog...,11 Psychological Effects of Being the Other Woman,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."


In [9]:
wikihow_df.tail(10)

,Soft Skill Name,Criteria,URL,Title of URL,Content
5,positive psychology,general guidelines,https://www.wikihow.com/Start-a-Private-Practi...,How to Start a Private Practice in Psychology ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
6,positive psychology,general guidelines,https://www.wikihow.com/Do-Reverse-Psychology-...,10 Easy Ways to Do Reverse Psychology on a Stu...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
7,positive psychology,general guidelines,https://www.wikihow.com/Get-Your-Man-Back-with...,How to Get Your Man Back with Reverse Psycholo...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
8,positive psychology,general guidelines,https://www.wikihow.com/Overcome-Psychological-ED,4 Effective Ways to Overcome Psychological ED ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
9,positive psychology,general guidelines,https://www.wikihow.com/Determine-if-a-wikiHow...,Log in to view,window.a=window.a||{};window.a.f=function(){fu...
10,positive psychology,general guidelines,https://www.wikihow.com/Pass-a-Psychological-T...,3 Ways to Pass a Psychological Test for a Job ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
11,positive psychology,general guidelines,https://www.wikihow.com/Write-a-Psychological-...,How to Write a Psychological Horror Story: 12 ...,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
12,positive psychology,general guidelines,https://www.wikihow.com/Category:Psychology-Ca...,Psychology Careers - how to articles from wikiHow,window.a=window.a||{};window.a.f=function(){fu...
13,positive psychology,general guidelines,https://www.wikihow.com/What-Are-the-Psycholog...,11 Psychological Effects of Being the Other Woman,"var dfpKeyVals = [];\nWH.gaType = ""mobile"";\nW..."
14,positive psychology,general guidelines,https://www.wikihow.com/Category:Psychology-St...,Psychology Studies - how to articles from wikiHow,window.a=window.a||{};window.a.f=function(){fu...


In [10]:
wikihow_df.to_excel('wiki_how_data_base.xlsx', index=False)
wikihow_df.to_csv('wiki_how_data_base.csv', index=False)

In [ ]:
#Let's check if we can get some paragraphs from Content
content_1 = list(wikihow_df['Content'])[50]
soup = bs4.BeautifulSoup(content_1, "html.parser")
paragraphs = soup.find_all('p')

In [ ]:
# print characters of first paragraph from 56 to 500
paragraphs[0].get_text()[56:500]